In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import Model, Sequential, models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

import sys
sys.path.append('../utility_packages/')

from image_utils import label_images, plot_images, count_samples

np.random.seed(6)

Using TensorFlow backend.


In [2]:
# define datasets and sample size
train_data_dir = "../data/seg_train/"
test_data_dir = "../data/seg_test/"

# define datasets and sample size
num_train_samples = count_samples(train_data_dir)
num_test_samples = count_samples(test_data_dir)

# define image dimensions
img_width, img_height = 150, 150
img_target_size = (img_width, img_height)

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
# define model hyperparameters
epochs = 100
batch_size = 64
neurons_per_layer = [64]
dropout = 0.50
n_classes = 6
patience = 3

## Model 1

In [4]:
model = Sequential()
model.add(layers.Conv2D(64, 3, input_shape=input_shape, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, strides=2, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, strides=2, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, strides=2, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(6, activation="softmax"))
model.summary()

W0817 16:01:55.271878 4490130880 deprecation.py:506] From /Users/ebolotin/Dropbox/techdev/env/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 144, 144, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 142, 142, 64)      36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 70, 70, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 68, 68, 128)       73856     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 33, 33, 128)       1

## Model 2

In [31]:
model = Sequential()
model.add(layers.Conv2D(128, 3, input_shape=input_shape, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(48, 3, activation="relu"))
model.add(layers.Conv2D(48, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(6, activation="softmax"))
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_310 (Conv2D)          (None, 148, 148, 128)     3584      
_________________________________________________________________
conv2d_311 (Conv2D)          (None, 146, 146, 128)     147584    
_________________________________________________________________
conv2d_312 (Conv2D)          (None, 144, 144, 128)     147584    
_________________________________________________________________
conv2d_313 (Conv2D)          (None, 142, 142, 96)      110688    
_________________________________________________________________
conv2d_314 (Conv2D)          (None, 140, 140, 96)      83040     
_________________________________________________________________
conv2d_315 (Conv2D)          (None, 138, 138, 96)      83040     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 69, 69, 96)      

## Model 3

In [6]:
# specify base model and freeze weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# create model function
def create_model(base_model, dropout, neurons_per_layer, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = layers.Flatten()(x)
    
    for num_neurons in neurons_per_layer:
        x = layers.Dense(num_neurons, activation='relu')(x) 
    
    x = layers.Dropout(dropout)(x)

    classifications = layers.Dense(num_classes, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=classifications)
    
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])

    return model

# instantiate model
model = create_model(base_model, dropout=dropout, neurons_per_layer=neurons_per_layer, num_classes=n_classes)
model.summary()

/Users/ebolotin/Dropbox/techdev/env/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________